In [1]:
from work.unet_main import *
from work.unet import unet_model_functions

from keras.callbacks import ReduceLROnPlateau,EarlyStopping


Using TensorFlow backend.


In [2]:
train_path=r'D:\Clarifruit\cherry_stem\data\raw_data\with_maskes'
dest_path=r'D:\Clarifruit\cherry_stem\data\unet_data\training'
test_path=r'D:\Clarifruit\cherry_stem\data\raw_data\images_orig'
src_path=r'D:\Clarifruit\cherry_stem\data\unet_data\training\2019-10-11_18-09-39'



reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.2,
                              patience=2, min_lr=0.000001,
                              cooldown=1, verbose=1)

early_stoping = EarlyStopping(monitor='val_loss',
                              min_delta=0,
                              patience=3,
                              verbose=0,
                              mode='auto',
                              baseline=None,
                              restore_best_weights=False)

callbacks = [reduce_lr,early_stoping]


In [6]:
#from scratch
params_dict = dict(

    train_path=train_path,
    x_folder_name='image',
    y_folder_name='label',
    weights_file_name='unet_cherry_stem.hdf5',

    data_gen_args=dict(rescale=1. / 255,
                       rotation_range=180,
                       brightness_range=[0.2, 1.],
                       width_shift_range=0.25,
                       height_shift_range=0.25,
                       shear_range=0.2,
                       zoom_range=0.5,
                       horizontal_flip=True,
                       vertical_flip=True,
                       featurewise_center=True,
                       featurewise_std_normalization=True,
                       zca_whitening=True,
                       channel_shift_range=50,
                       fill_mode='nearest'),

    optimizer='Adam',
    optimizer_params=dict(lr=1e-4),
    loss='binary_crossentropy',
    metrics=['accuracy'],
    pretrained_weights=None,

    target_size=(256, 256),
    color_mode='rgb',
    batch_size=10,
    epochs=5,
    steps_per_epoch=1000,
    valdiation_split=0.2,
    validation_steps=200)

params_dict['callbacks'] = callbacks

In [18]:
#load pretrained
params_dict = unet_model_functions.ClarifruitUnet.load_model(src_path)
params_dict['callbacks'] = callbacks
params_dict['metrics']=['accuracy']

In [13]:
train_params = dict(

    data_gen_args=dict(rescale=1. / 255,
                       rotation_range=180,
                       brightness_range=[0.2, 1.],
                       width_shift_range=0.25,
                       height_shift_range=0.25,
                       shear_range=0.2,
                       zoom_range=0.5,
                       horizontal_flip=True,
                       vertical_flip=True,
                       fill_mode='nearest',
                       zca_whitening=True,
                       featurewise_center=True,
                       featurewise_std_normalization=True,
                       channel_shift_range=50),

    callbacks=[reduce_lr],

    batch_size=10,
    epochs=3,
    steps_per_epoch=3000,
    valdiation_split=0.2,
    validation_steps=300)

params_dict.update(train_params)  # uncomment to modify the model parameters

In [7]:
model = unet_model_functions.ClarifruitUnet(**params_dict)
model.train_model(dest_path=dest_path,params_dict=params_dict)

2019-10-12 11:54:17,357-6d  INFO     work.unet.unet_model: <- unet model with input_size=(256, 256, 3) andpretraind_weights=None 


d:\clarifruit\cherry_stem\cherry_venv\lib\site-packages\keras_preprocessing\image\image_data_generator.py:341: UserWarning: This ImageDataGenerator specifies `zca_whitening` which overrides setting of`featurewise_std_normalization`.
  warnings.warn('This ImageDataGenerator specifies '


Found 152 images belonging to 1 classes.
Found 38 images belonging to 1 classes.
Found 152 images belonging to 1 classes.
Found 38 images belonging to 1 classes.
Epoch 1/5


ResourceExhaustedError: OOM when allocating tensor with shape[10,128,256,256] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node training_1/Adam/gradients/conv2d_45/convolution_grad/Conv2DBackpropInput}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


In [5]:
model.prediction(test_path,dest_path)


2019-10-11 16:40:40,096-6d  INFO     work.unet.unet_model_functions: prediction on files from D:\Clarifruit\cherry_stem\data\raw_data\images_orig
2019-10-11 16:40:40,099-6d  INFO     work.unet.unet_model_functions: saving predictions to D:\Clarifruit\cherry_stem\data\unet_data\training\2019-10-11_13-59-38\raw_pred


'D:\\Clarifruit\\cherry_stem\\data\\unet_data\\training\\2019-10-11_13-59-38\\raw_pred'

In [2]:
from work.stem_classifier.model_functions import *


In [ ]:
train_path=r'D:\Clarifruit\cherry_stem\data\classification_data\from_all\set1\train'
dest_path=r'D:\Clarifruit\cherry_stem\data\unet_data\training\2019-10-07_20-12-39'
test_path=r'D:\Clarifruit\cherry_stem\data\classification_data\from_all\set1\test'

params_dict = dict(

    train_path=train_path,

    data_gen_args=dict(rescale=1. / 255,
                       rotation_range=180,
                       width_shift_range=0.25,
                       height_shift_range=0.25,
                       shear_range=0.2,
                       zoom_range=[0.5, 1.0],
                       horizontal_flip=True,
                       vertical_flip=True,
                       fill_mode='nearest'),

    optimizer='Adam',
    optimizer_params=dict(lr=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy'],
    pretrained_weights=None,

    target_size=(256, 256),
    color_mode='rgb',
    batch_size=10,
    epochs=10,
    steps_per_epoch=3000,
    valdiation_split=0.2,
    validation_steps=300)



model = ClarifruitClassifier(**params_dict)

model.train_model(dest_path=dest_path,params_dict=params_dict)




Found 349 images belonging to 4 classes.
Found 85 images belonging to 4 classes.
Epoch 1/10
3000/3000 [==============================] - 994s 331ms/step - loss: 1.1869 - acc: 0.4543 - val_loss: 2.3112 - val_acc: 0.3619

Epoch 00001: loss improved from inf to 1.18693, saving model to D:\Clarifruit\cherry_stem\data\unet_data\training\2019-10-07_20-12-39\2019-10-11_03-08-49\model_weights.hdf5
Epoch 2/10
3000/3000 [==============================] - 1050s 350ms/step - loss: 0.6502 - acc: 0.7206 - val_loss: 3.8966 - val_acc: 0.3954

Epoch 00002: loss improved from 1.18693 to 0.65009, saving model to D:\Clarifruit\cherry_stem\data\unet_data\training\2019-10-07_20-12-39\2019-10-11_03-08-49\model_weights.hdf5
Epoch 3/10
 448/3000 [===>..........................] - ETA: 13:08 - loss: 0.5087 - acc: 0.7811

In [ ]:
model.prediction(test_path,dest_path)